# Данный файл предназначен для сравнения АДХ летательного аппарата рассчитанных в класс LA с полученными в Ansys
# НЕ РАБОТАЕТ, НЕ СДЕЛАНО!

In [1]:
import os
import sys
AeroBDSM_OOP_dir = os.path.abspath(os.path.join('../../..'))
if AeroBDSM_OOP_dir not in sys.path:
    sys.path.append(AeroBDSM_OOP_dir)

from fuselage import *
from isolated_plane import *
from __LA import *
import numpy as np
from scipy.interpolate import interp2d
import matplotlib.pyplot as plt
from ipywidgets import interact

%config InlineBackend.figure_format = 'svg'

#параметры текста
plt.rcParams["font.family"] = 'serif'
plt.rcParams['font.weight'] = 'ultralight'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 14
plt.rcParams['font.style'] = 'normal'
plt.rcParams['mathtext.default'] = 'regular'
plt.rcParams['mathtext.fontset'] = 'stix'

ModuleNotFoundError: No module named 'fuselage'

In [ ]:
alphas = np.deg2rad([0,2,5,10,15,20])
Ms = [0.6, 0.8, 1.1, 1.5, 2]

Y_ansysm = [[-2.4778, 470.48, 1410.58, 3464.6, 5964.4, 8702],
            [-3.134, 834.936, 2544.2, 6348.8, 10938.4, 15928.9],
            [4.5416, 1604.924, 5140.473, 12682.769, 21716.683, 31986.954],
            [-4.217, 3203.403, 9600.985, 23681.586, 40631.119, 62924.287],
            [-17.476, 5525.547, 16452.775, 40970.1, 71439.856, 112187.357]]

X_ansysm = [[589.88, 596.9, 618.8, 629.44, 573.58, 493.52],
            [1080.24, 1087.85, 1125.10, 1143.32, 1048.49, 930.08],
            [4683, 4688.208, 4750.958, 4949.38, 4917.055, 4667.264],
            [9120.703, 9144.984, 9260.561, 9524.356, 9738.117, 9745.524],
            [14690.643, 14784.394, 15055.862, 15489.212, 16068.561, 16129.074]]

Mz_ansysm = [[-11.668, 932.86, 2993.4, 7688, 13468.2, 19783.4],
             [-15.99, 1627.62, 5392.27, 14112, 24837.6, 36238.9],
             [1.3016, 3352.16, 10857.473, 28134.45, 48411.16, 69238.992],
             [-68.225, 5876.737, 19639.399, 51461.907, 87267.676, 127955.5],
             [-92.364, 9848.47, 32496.898, 85716.655, 142648.235, 225990.819]]

S_har = 0.493
b_har = 0.897
x_mass = 4.2 * 0.529

In [ ]:
alpham, Mm = np.meshgrid(alphas, Ms)
_Y_ansys = interp2d(alpham, Mm, Y_ansysm)
_X_ansys = interp2d(alpham, Mm, X_ansysm,)
_Mz_ansys1 = interp2d(alpham, Mm, Mz_ansysm)

Y_ansys = lambda M, alpha: _Y_ansys(alpha, M)
X_ansys = lambda M, alpha: _X_ansys(alpha, M)
Mz_ansys1 = lambda M, alpha: _Mz_ansys1(alpha, M)
Mz_ansys = lambda M, alpha: Y_ansys(M, alpha) * x_mass - Mz_ansys1(M ,alpha)

cx_ansys = lambda M, alpha: X_ansys(M, alpha) / (q(M, 0) * S_har)
cy_ansys = lambda M, alpha: Y_ansys(M, alpha) / (q(M, 0) * S_har)
mz_ansys = lambda M, alpha: Mz_ansys(M, alpha) / (q(M, 0) * S_har * b_har)
xd_ansys = lambda M, alpha: Mz_ansys1(M, alpha) / Y_ansys(M, alpha)

cya_ansys = lambda M, alpha: cy_ansys(M, alpha) * np.cos(alpha) - cx_ansys(M, alpha) * np.sin(alpha)
cxa_ansys = lambda M, alpha: cy_ansys(M, alpha) * np.sin(alpha) + cx_ansys(M, alpha) * np.cos(alpha)

In [ ]:
'''Фюзеляж'''
D = 0.31
L_main = 4.2
L_nos = 0.37
L_cor = 0.115
D_dn = 0.27
r_sph = 0.5 * D / 2

lambda_nos = L_nos / D
lambda_cor = L_cor / D
lambda_cil = (L_main - L_cor - L_nos) / D
eta_cor = D_dn / D

nc = NosCil_SphOgi(lambda_nos=lambda_nos,
                   lambda_cil=lambda_cil,
                   D=D,
                   r_sph=r_sph)
cor = Cor_Con(D=D,
              eta_cor=eta_cor,
              lambda_cor=lambda_cor,
              D_a=(0.9 * D_dn))
fuselag = Fuselage(nosCil=nc, cor=cor)

In [ ]:
'''Передняя НП'''
x_bb_I = 1.94
l_c_I = 0.248                               # длина консоли [м]
b_b_I = 0.897                               # бортовая хорда [м]
b_1_I = 0.773                               # концевая хорда [м]
chi_0_I = radians(45)                       # угол стреловидности по передней кромке крыла [рад]

# параметры профиля
cc_1_I = 0.03                               # относительная толщина профиля в концевом сечении
cc_b_I = 0.11                               # относительная толщина профиля в бортовом сечении
cc_I = (cc_b_I + cc_1_I) / 2                # средняя относительная толщина профиля [-]
xx_c_I = 0.5                               # относительная координата наибольшей толщины профиля в долях рассматриваемой хорды
rr_0_I = 0                                  # относительный радиус скругления передней кромки [-]
hb_I = 1e-5                                 # относительная высота бугорков профиля [-]
aa_I = 0.5                                  # отношение геометрического параметра профиля `a` в долях рассматриваемой хорды

S_c_I = (b_b_I + b_1_I) / 2 * l_c_I
lambda_c_I = l_c_I ** 2 / S_c_I
zeta_c_I = b_1_I / b_b_I
chi_05_I = atan(tan(chi_0_I) - 2 / lambda_c_I * (1 / zeta_c_I - 1) / (1 / zeta_c_I + 1))

prof1 = HexProfile(b_b_I, cc_I, xx_c_I, aa_I, rr_0_I, hb_I)
IC_I = IsolatedPlane(S_c_I, lambda_c_I, zeta_c_I, chi_05_I, prof1)

In [ ]:
# Геометрические задней НП

x_bb_II = 3.85
# Параметры крыла
l_kr_II = 0.248            # размах крыла [м]
l_c_II = 0.124             # длина одной консоли [м]
b_b_II = 0.340             # бортовая хорда [м]
b_1_II = 0.200             # концевая хорда [м]
chi_0_II = radians(48.47)  # угол стреловидности по передней кромке [рад]
chi_1_II = radians(0)      # угол стреловидности по задней кромке [рад]
x_b_II = 2.200             # координата носка бортовой хорды (от носка самолета) [м]
xx_r_II = 0.6              # относительная координата оси вращения руля

# Параметры профиля крыла
cc_b_II = 0.011            # относительная толщина профиля бортовой хорды [-]
cc_1_II = 0.030            # относительная толщина профиля концевой хорды [-]
cc_II = (cc_b_II + cc_1_II) / 2  # средняя относительная толщина профиля [-]
xx_c_II = 0.5              # относительная координата положения максимальной толщины [-]
hb_II = 1e-5               # относительная высота бугорков профиля [-]
rr_0_II = 0                # относительный радиус скругления передней кромки [-]
aa_II = 0.5                # отношение геометрического параметра профиля `a` в долях рассматриваемой хорды [-]

# Расчет геометрических параметров
S_kr_II = (b_b_II + b_1_II) / 2 * l_kr_II
lambda_kr_II = l_kr_II ** 2 / S_kr_II
zeta_kr_II = b_1_II / b_b_II
chi_05_II = atan(tan(chi_0_II) - 2 / lambda_kr_II * (1 / zeta_kr_II - 1) / (1 / zeta_kr_II + 1))

prof2 = HexProfile(b_b_II, cc_II, xx_c_II, aa_II, rr_0_II, hb_II)
IC_II = IsolatedPlane_RotatingConsoles(S_kr_II, lambda_kr_II, zeta_kr_II, chi_05_II, S_kr_II, lambda_kr_II, xx_r_II, prof2)

In [ ]:
console_row_1 = [
    [x_bb_I, np.radians([45, 135, 225, 315]), IC_I]
]

console_row_2 = [
    [x_bb_II, np.radians([45, 135, 225, 315]), IC_II]
]

la = LA(fuselag, console_row_1, console_row_2, S_har, b_har)

In [ ]:
deltas_I = [0, 0, 0, 0]
deltas_II = [0, 0, 0, 0]

def coeff_M(alpha):
    alpha_r = np.deg2rad(alpha)
    
    fig, axs = plt.subplots(2, 3, figsize = (16, 9))
    axs = np.reshape(axs,(1,6))[0]
    
    c_ys = [la.c_y(Mi, alpha_r, deltas_I, deltas_II, 0) for Mi in Ms]
    c_xas = [la.c_xa(Mi, alpha_r, 0, deltas_I, deltas_II, False, 0) for Mi in Ms]
    c_yas = [la.c_ya(Mi, alpha_r, deltas_I, deltas_II, False, 0) for Mi in Ms]
    m_zs = [la.m_z(Mi, alpha_r, deltas_I, deltas_II, x_mass, 0) for Mi in Ms]
    x_ds = [la.x_d(Mi, alpha_r, deltas_I, deltas_II, 0) for Mi in Ms]

    labels = ['$c_y$','$c_{xa}$', '$c_{ya}$','$m_z$','$x_d$']

    for ax, c, c_ansys, label in zip(axs, [c_ys, c_xas, c_yas, m_zs, x_ds], [cy_ansys, cxa_ansys, cya_ansys, mz_ansys, xd_ansys], labels):
        ax.plot(Ms, c, label = 'ООП', c = 'r')
        if alpha > 0 or c_ansys != xd_ansys:
            ax.plot(Ms, [c_ansys(Mi, alpha_r) for Mi in Ms], label = 'ANSYS', c = 'b')

        ax.set_xlabel('M')
        ax.set_ylabel(label)
        ax.grid()
        ax.legend()
    plt.show()

interact(coeff_M, alpha = np.rad2deg(alphas))

interactive(children=(Dropdown(description='alpha', options=(0.0, 2.0, 5.0, 10.0, 14.999999999999998, 20.0), v…

<function __main__.coeff_M(alpha)>

In [ ]:
def coeff_alpha(M):
    alphas_deg = np.rad2deg(alphas)
    
    fig, axs = plt.subplots(2, 3, figsize = (16, 9))
    axs = np.reshape(axs,(1,6))[0]
    
    c_ys = [la.c_y(M, alpha, deltas_I, deltas_II, 0) for alpha in alphas]
    c_xas = [la.c_xa(M, alpha, 0, deltas_I, deltas_II, False, 0) for alpha in alphas]
    c_yas = [la.c_ya(M, alpha, deltas_I, deltas_II, False, 0) for alpha in alphas]
    m_zs = [la.m_z(M, alpha, deltas_I, deltas_II, x_mass, 0) for alpha in alphas]
    x_ds = [la.x_d(M, alpha, deltas_I, deltas_II, 0) for alpha in alphas]

    labels = ['$c_y$','$c_{xa}$', '$c_{ya}$','$m_z$','$x_d$']

    for ax, c, c_ansys, label in zip(axs, [c_ys, c_xas, c_yas, m_zs, x_ds], [cy_ansys, cxa_ansys, cya_ansys, mz_ansys, xd_ansys], labels):
        ax.plot(alphas_deg, c, label = 'ООП', c = 'r')
        if c_ansys == xd_ansys:
            ax.plot(alphas_deg[1:], [c_ansys(M, alpha) for alpha in alphas[1:]], label = 'ANSYS', c = 'b')
        else:
            ax.plot(alphas_deg, [c_ansys(M, alpha) for alpha in alphas], label = 'ANSYS', c = 'b')

        ax.set_xlabel('$\\alpha$')
        ax.set_ylabel(label)
        ax.grid()
        ax.legend()
    plt.show()

interact(coeff_alpha, M = Ms)

interactive(children=(Dropdown(description='M', options=(0.6, 0.8, 1.1, 1.5, 2), value=0.6), Output()), _dom_c…

<function __main__.coeff_alpha(M)>